In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install  -q accelerate
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login
from google.colab import userdata

In [ ]:
hf_api = userdata.get('HF_API_KEY')

In [ ]:
login(token = hf_api)

In [ ]:
model_name = ("mistralai/Mistral-7B-Instruct-v0.2")

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map = "auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:


def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    attention_mask = inputs["attention_mask"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask = attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        num_beams=4,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=100
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        return output.split("### Response:")[1].strip()

In [ ]:
des = "During their decade-long quest to defeat the Demon King, the members of the hero's party—Himmel himself, the priest Heiter, the dwarf warrior Eisen, and the elven mage Frieren—forge bonds through adventures and battles, creating unforgettable precious memories for most of them. However, the time that Frieren spends with her comrades is equivalent to merely a fraction of her life, which has lasted over a thousand years. When the party disbands after their victory, Frieren casually returns to her 'usual' routine of collecting spells across the continent. Due to her different sense of time, she seemingly holds no strong feelings toward the experiences she went through. As the years pass, Frieren gradually realizes how her days in the hero's party truly impacted her. Witnessing the deaths of two of her former companions, Frieren begins to regret having taken their presence for granted; she vows to better understand humans and create real personal connections. Although the story of that once memorable journey has long ended, a new tale is about to begin."
allq= evaluate(f"give me 1 natural language query that you think user will input it for query search, without reutrning the prompt and opinion on query and return only the query if the title or title synonyms appeared in the description just ignore it, which don't be too specific and not mention about anime name and character that will return a anime with the following description :\n{des} don't return the note and question after the query")
print(allq)

What is the experience of an immortal mage, who spent a short time with a hero's party during their quest against the Demon King, and how does she come to value her human connections?</s>


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/anime_recommend_data_real.csv')

In [ ]:
df.head()

,mal_id,url,title,image_jpg_url,type,episodes,rating,score,synopsis,genres,q
0,52991,https://myanimelist.net/anime/52991/Sousou_no_...,Sousou no Frieren,https://cdn.myanimelist.net/images/anime/1015/...,TV,28.0,PG-13 - Teens 13 or older,9.30,During their decade-long quest to defeat the D...,"Adventure, Drama, Fantasy",
1,5114,https://myanimelist.net/anime/5114/Fullmetal_A...,Fullmetal Alchemist: Brotherhood,https://cdn.myanimelist.net/images/anime/1208/...,TV,64.0,R - 17+ (violence & profanity),9.10,After a horrific alchemy experiment goes wrong...,"Action, Adventure, Drama, Fantasy",
2,9253,https://myanimelist.net/anime/9253/Steins_Gate,Steins;Gate,https://cdn.myanimelist.net/images/anime/1935/...,TV,24.0,PG-13 - Teens 13 or older,9.07,Eccentric scientist Rintarou Okabe has a never...,"Drama, Sci-Fi, Suspense",
3,38524,https://myanimelist.net/anime/38524/Shingeki_n...,Shingeki no Kyojin Season 3 Part 2,https://cdn.myanimelist.net/images/anime/1517/...,TV,10.0,R - 17+ (violence & profanity),9.05,Seeking to restore humanity's diminishing hope...,"Action, Drama, Suspense",
4,28977,https://myanimelist.net/anime/28977/Gintama°,Gintama°,https://cdn.myanimelist.net/images/anime/3/720...,TV,51.0,PG-13 - Teens 13 or older,9.05,"Gintoki, Shinpachi, and Kagura return as the f...","Action, Comedy, Sci-Fi",


In [ ]:
df.loc[2009,"q"]

'"Show me an anime about a young boy growing up along a river in the mid 1800s, having adventures with his best friend."</s>'

In [ ]:
%%time
for i in range(2100, len(df)+1):
  des = df.loc[i, "synopsis"]
  name_s = df.loc[i, "title"]
  allq= evaluate(f"give me 1 natural language query that you think user will input it for query search, without reutrning the prompt and opinion on query and return only the query, which don't be too specific and not mention about anime name and character that will return a anime with the following description :\n{des} don't return the note and question after the query")
  df.loc[i, "q"] = allq
  print(i, name_s, "done.")
df.to_csv('anime_recommend_data_real.csv', index = False)
!cp anime_recommend_data_real.csv '/content/drive/MyDrive'

In [ ]:
df.loc[2500, "q"]

'What is the story about a girl with a wild running style in a town known for horse girls and running prodigies?</s>'

In [ ]:
def clean_end_mark(text):
  return text.replace('</s>', '')

In [ ]:
df['q'] = df['q'].apply(clean_end_mark)
df = df.sort_values(by='score', ascending = False)
df = df.reset_index(drop = True)
df

,mal_id,url,title,image_jpg_url,type,episodes,rating,score,synopsis,genres,q
0,52991,https://myanimelist.net/anime/52991/Sousou_no_...,Sousou no Frieren,https://cdn.myanimelist.net/images/anime/1015/...,TV,28.0,PG-13 - Teens 13 or older,9.30,During their decade-long quest to defeat the D...,"Adventure, Drama, Fantasy","""What is a story about a group of heroes on a ..."
1,5114,https://myanimelist.net/anime/5114/Fullmetal_A...,Fullmetal Alchemist: Brotherhood,https://cdn.myanimelist.net/images/anime/1208/...,TV,64.0,R - 17+ (violence & profanity),9.10,After a horrific alchemy experiment goes wrong...,"Action, Adventure, Drama, Fantasy",What is a story about two brothers who undergo...
2,9253,https://myanimelist.net/anime/9253/Steins_Gate,Steins;Gate,https://cdn.myanimelist.net/images/anime/1935/...,TV,24.0,PG-13 - Teens 13 or older,9.07,Eccentric scientist Rintarou Okabe has a never...,"Drama, Sci-Fi, Suspense",What is a query for an anime about a group of ...
3,28977,https://myanimelist.net/anime/28977/Gintama°,Gintama°,https://cdn.myanimelist.net/images/anime/3/720...,TV,51.0,PG-13 - Teens 13 or older,9.05,"Gintoki, Shinpachi, and Kagura return as the f...","Action, Comedy, Sci-Fi",What can Gintoki and his team be seen doing in...
4,38524,https://myanimelist.net/anime/38524/Shingeki_n...,Shingeki no Kyojin Season 3 Part 2,https://cdn.myanimelist.net/images/anime/1517/...,TV,10.0,R - 17+ (violence & profanity),9.05,Seeking to restore humanity's diminishing hope...,"Action, Drama, Suspense","""What happens when the Survey Corps return to ..."
...,...,...,...,...,...,...,...,...,...,...,...
2496,4703,https://myanimelist.net/anime/4703/Yes_Precure...,Yes! Precure 5 GoGo! Movie: Okashi no Kuni no ...,https://cdn.myanimelist.net/images/anime/1889/...,Movie,1.0,G - All Ages,7.17,"It's Nozomi's birthday, but it is disturbed by...",Romance,What is the story about when Nozomi's birthday...
2497,3692,https://myanimelist.net/anime/3692/Yes_Precure...,Yes! Precure 5 GoGo!,https://cdn.myanimelist.net/images/anime/1149/...,TV,48.0,G - All Ages,7.17,"Nozomi and friends are back, their role as Pre...",Action,"""How do Nozomi and her friends find and use th..."
2498,58271,https://myanimelist.net/anime/58271/Honey_Lemo...,Honey Lemon Soda,https://cdn.myanimelist.net/images/anime/1382/...,TV,12.0,PG-13 - Teens 13 or older,7.17,First-year high school student Uka Ishimori wa...,Romance,"""A shy and introverted first-year high school ..."
2499,94,https://myanimelist.net/anime/94/Kidou_Senshi_...,Kidou Senshi Gundam SEED Destiny,https://cdn.myanimelist.net/images/anime/1700/...,TV,50.0,R - 17+ (violence & profanity),7.17,"In the year Cosmic Era 0071, a wartime brawl b...","Action, Drama, Romance, Sci-Fi","""What is a story about a young man seeking rev..."


In [ ]:
df.to_csv('anime_recommend_data_real.csv', index = False)
!cp anime_recommend_data_real.csv '/content/drive/MyDrive'

In [ ]:
from google.colab import runtime
runtime.unassign()